# スマート マッピング
スマート マッピングは、ArcGIS Online および Portal for ArcGIS (10.3.1 以降) に組み込まれた新しい機能で、美しいマップを簡単に作成することができます。スマート マッピングを使用すると、データを素早く分析し、デフォルトを設定 (シンボルの大きさや、色、表示縮尺、分類の閾値などが自動で設定) して、簡単にイラストマップを作成することができます。

このエキサイティングな機能の詳細については、[ヘルプページ](http://www.esri.com/software/arcgis/smart-mapping)と[このブログ記事](https://blogs.esri.com/esri/arcgis/2015/03/02/introducing-smart-mapping/)をご覧ください。ここには、データのスタイルを変更する様々な方法についての [詳細なヘルプ](http://doc.arcgis.com/en/arcgis-online/create-maps/change-style.htm)があります。

## ラインの可視化
最初の例では、streets レイヤーを使用して直線的な特徴を可視化する方法を示しています。マップに `Item` オブジェクトを追加すると、マップウィジェットはアイテムのデフォルトのシンボルを使用してレンダリングします。スマートマッピングを使えば、これをカスタマイズすることができます。この例では、**ClassSizeRenderer** を使用して、ライン フィーチャの太さを変化させて値の違いを強調しています。

In [36]:
from arcgis.gis import *
gis = GIS()

In [37]:
map1 = gis.map('USA',3)
map1

MapView(layout=Layout(height='400px', width='100%'), zoom=3.0)

**USA Freeway System** by **Esri** というタイトルの項目を検索して、地図に追加してみましょう。

In [38]:
search_result = gis.content.search('title:USA Freeway system AND owner:esri_dm', 
                                   item_type = 'Feature Layer')
search_result

[<Item title:"USA Freeway System" type:Feature Layer Collection owner:esri_dm>,
 <Item title:"USA Freeway System" type:Feature Layer Collection owner:esri_dm>]

In [39]:
freeway_item = search_result[0]
map1.add_layer(freeway_item)
map1

MapView(jupyter_target='notebook', layout=Layout(height='400px', width='100%'), ready=True, zoom=3.0)

 高速道路 (freeway) レイヤーはシンプルなシンボルを使用しています。次にフィーチャ レイヤーの利用可能なフィールドの1つで可視化してみましょう。そのためには、アイテムのレイヤーを **FeatureLayer** オブジェクトとして取得します。オブジェクトの `FeatureLayer` から、URL と利用可能なフィールドを見つけることができます。

In [40]:
# サービスで利用可能な1番目のレイヤーを使用します.
freeway_feature_layer = freeway_item.layers[0]

# レイヤーの URL とフィールド名を表示します.
print(freeway_feature_layer.url)
for field in freeway_feature_layer.properties['fields']:
    print(field['name'])

https://services.arcgis.com/P3ePLMYs2RVChkJx/arcgis/rest/services/USA_Freeway_System/FeatureServer/1
OBJECTID
ROUTE_NUM
CLASS
NUMBER
SUFFIX
DIST_MILES
DIST_KM


ここでは `DIST_MILES` フィールドを使用して、さまざまな色を指定してシンボルを設定してみましょう。また、地図の範囲を変えて、違いをより分かりやすくしましょう。

In [41]:
map1.add_layer(freeway_feature_layer, {"renderer":"ClassedSizeRenderer", "field_name": "DIST_MILES"})

In [44]:
# 地図の範囲をロサンゼルスに変更する
map1.center = [34, -118]

In [45]:
map1.zoom = 10
map1

MapView(heading=359.9999931698108, jupyter_target='notebook', layout=Layout(height='400px', width='100%'), rea…

このように、長い距離の高速道路は太い線で、短い距離の高速道路は細い線で表現しました。

## ポリゴンの可視化

エリアまたはポリゴン フィーチャは、値の違いを表すために様々な色で表示されます。
以下の例は、**ClassedColorRenderer** を使用して、ワシントン州の人口の違いを可視化する方法を示しています。

前のサンプルで見たように、**スマートマッピング** の機能を使用することで、レンダラーのタイプと使用するフィールドを指定するだけでマップを作成でき、あとはベースマップに基づいて適切なカラースキーム (配色)を特定したり、カラーランプの最小値と最大値を指定したりします。

また、定義式を使ってレイヤーから表示されるフィーチャを制限する方法や、ベースマップが見えるようにレイヤーを透過度で表現する方法も紹介しています。

In [60]:
map2 = gis.map('Seattle, WA', 6)

In [61]:
map2

MapView(layout=Layout(height='400px', width='100%'), zoom=6.0)

フィーチャ レイヤーの項目を検索してその URL を取得する手順は、前回のサンプルで説明しました。簡潔にするために、このサンプルではその部分を省略しています。

In [62]:
map2.add_layer({"type":"FeatureLayer", 
               "url":"//sampleserver6.arcgisonline.com/arcgis/rest/services/Census/MapServer/2",
               "definition_expression" : "STATE_NAME='Washington'",
               "renderer":"ClassedColorRenderer",
               "field_name":"POP2007",
               "opacity":0.7
              })
map2

MapView(jupyter_target='notebook', layout=Layout(height='400px', width='100%'), ready=True, zoom=6.0)

## 位置情報の可視化
ポイント フィーチャは、最も一般的なタイプの位置データです。スマートマッピングは、**heatmap (ヒートマップ)** と呼ばれる特殊な可視化手法を提供しています。 ヒートマップ レンダラーは、マップ上のポイントの相対密度を、cool (低密度) から hot (多数のポイント) までの滑らかに変化する色のセットとして表現するため、ポイントの空間的な分布やクラスタリングを表現する場合に便利です。

以下のサンプルでは、ヒートマップ レンダラーを使用して、南カリフォルニアで発生した地震を可視化しています。

In [12]:
map3 = gis.map('Los Angeles', 8)

In [13]:
map3

MapView(layout=Layout(height='400px', width='100%'), zoom=8.0)

![location data](http://esri.github.io/arcgis-python-api/notebooks/nbimages/02_SmartMapping_03.PNG)

In [14]:
map3.add_layer({"type":"FeatureLayer",
                "url":"http://services1.arcgis.com/hLJbHVT9ZrDIzK0I/arcgis/rest/services/EQMagGt4/FeatureServer/0",
                "renderer":"HeatmapRenderer",
                "opacity":0.75})